In [1]:
!pip install --upgrade spleeter noisereduce librosa soundfile pydub tensorflow tensorflow-estimator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 82.1 MB/s eta 0:0

In [1]:
import os
import glob
import tempfile
from spleeter.separator import Separator
import noisereduce as nr
import librosa
import soundfile as sf
from pydub import AudioSegment
from pydub.effects import low_pass_filter, high_pass_filter
from tqdm import tqdm

def convert_to_wav(input_path, output_path):
    """Конвертирует аудиофайл в WAV формат"""
    try:
        audio = AudioSegment.from_file(input_path)
        audio.export(output_path, format="wav")
        return output_path
    except Exception as e:
        print(f"Error converting {input_path}: {str(e)}")
        return None

def separate_vocals(input_audio, output_path):
    """Отделяет вокал от музыки с помощью Spleeter"""
    try:
        separator = Separator('spleeter:2stems')
        separator.separate_to_file(input_audio, output_path)
        base_name = os.path.splitext(os.path.basename(input_audio))[0]
        return os.path.join(output_path, base_name, "vocals.wav")
    except Exception as e:
        print(f"Error separating vocals from {input_audio}: {str(e)}")
        return None

def clean_audio(input_path, output_path, noise_start=0, noise_end=1):
    """Очищает аудио от шумов"""
    try:
        y, sr = librosa.load(input_path, sr=None)
        noise_sample = y[int(noise_start * sr) : int(noise_end * sr)]
        cleaned_audio = nr.reduce_noise(y=y, sr=sr, y_noise=noise_sample, stationary=True, prop_decrease=1.0)
        sf.write(output_path, cleaned_audio, sr)
        return output_path
    except Exception as e:
        print(f"Error cleaning {input_path}: {str(e)}")
        return None

def postprocess_audio(input_path, output_path):
    """Постобработка аудио"""
    try:
        audio = AudioSegment.from_wav(input_path)
        filtered = high_pass_filter(audio, cutoff=100)
        filtered = low_pass_filter(filtered, cutoff=4000).normalize()
        filtered.export(output_path, format="wav")
        return output_path
    except Exception as e:
        print(f"Error postprocessing {input_path}: {str(e)}")
        return None

def process_single_audio(input_path, output_folder):
    """Обрабатывает один аудиофайл с полной очисткой временных файлов"""
    try:
        base_name = os.path.splitext(os.path.basename(input_path))[0]
        final_path = os.path.join(output_folder, f"{base_name}_clean.wav")

        with tempfile.TemporaryDirectory() as temp_dir:
            temp_wav = os.path.join(temp_dir, f"{base_name}.wav")
            if not convert_to_wav(input_path, temp_wav):
                return None

            vocals_path = separate_vocals(temp_wav, temp_dir)
            if not vocals_path or not os.path.exists(vocals_path):
                return None

            cleaned_path = os.path.join(temp_dir, f"cleaned_{base_name}.wav")
            if not clean_audio(vocals_path, cleaned_path):
                return None

            if not postprocess_audio(cleaned_path, final_path):
                return None

        return final_path
    except Exception as e:
        print(f"Error processing {input_path}: {str(e)}")
        return None

def process_audio_folder(input_folder, output_folder):
    """Обрабатывает все аудиофайлы в папке"""
    os.makedirs(output_folder, exist_ok=True)
    audio_files = []

    for ext in ['*.mp3', '*.wav', '*.WAV', '*.MP3']:
        audio_files.extend(glob.glob(os.path.join(input_folder, ext)))

    if not audio_files:
        print(f"No audio files found in {input_folder}")
        return []

    print(f"Found {len(audio_files)} audio files to process")
    processed_files = []

    for audio_file in tqdm(audio_files, desc="Processing audio files"):
        result = process_single_audio(audio_file, output_folder)
        if result:
            processed_files.append(result)

    print(f"\nProcessing complete! {len(processed_files)} files processed")
    print(f"Cleaned files saved to: {output_folder}")
    return processed_files

input_folder = "/content/input_audio"
output_folder = "/content/cleaned_audio"

os.makedirs(input_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

processed_files = process_audio_folder(input_folder, output_folder)

Found 20 audio files to process


Processing audio files:   0%|          | 0/20 [00:00<?, ?it/s]

INFO:spleeter:Downloading model archive https://github.com/deezer/spleeter/releases/download/v1.4.0/2stems.tar.gz


INFO:spleeter:Downloading model archive https://github.com/deezer/spleeter/releases/download/v1.4.0/2stems.tar.gz


INFO:spleeter:Validating archive checksum


...

INFO:spleeter:File /tmp/tmpqmaxckq7/КСД - 11/vocals.wav written succesfully


INFO:spleeter:File /tmp/tmpqmaxckq7/КСД - 11/vocals.wav written succesfully


INFO:spleeter:File /tmp/tmpqmaxckq7/КСД - 11/accompaniment.wav written succesfully


INFO:spleeter:File /tmp/tmpqmaxckq7/КСД - 11/accompaniment.wav written succesfully
Processing audio files: 100%|██████████| 20/20 [15:54<00:00, 47.75s/it]


Processing complete! 20 files processed
Cleaned files saved to: /content/cleaned_audio


In [ ]:
import os

folder_path = "/content/cleaned_audio"

if not os.path.exists(folder_path):
    print("Error: Folder not found!")
    exit()

for num in range(1, 21):
    old_num = f"{num:02d}"
    new_num = f"{num:02d}"

    old_name = f"КСД - {old_num}_clean (mp3cut.net).wav"
    new_name = f"How_to_save_money_{new_num}.wav"

    old_path = os.path.join(folder_path, old_name)
    new_path = os.path.join(folder_path, new_name)

    if os.path.exists(old_path):
        os.rename(old_path, new_path)
        print(f"Renamed: {old_name} -> {new_name}")
    else:
        print(f"File not found: {old_name}")

print("Operation completed. Please verify the files.")

In [5]:
import pandas as pd
import os
import wave
from datetime import datetime

audio_folder = "/content/cleaned_audio"
output_csv = "audio_metadata.csv"
fixed_processing_date = datetime.now().strftime("%Y-%m-%d")

def get_wav_duration(filepath):
    """Get duration of WAV file in MM:SS format"""
    try:
        with wave.open(filepath, 'rb') as wav_file:
            frames = wav_file.getnframes()
            rate = wav_file.getframerate()
            duration = frames / float(rate)
            minutes = int(duration // 60)
            seconds = int(duration % 60)
            return f"{minutes:02d}:{seconds:02d}"
    except:
        return "00:00"

metadata = {
    "filename": [],
    "language": [],
    "speaker_gender": [],
    "duration": [],
    "topic": [],
    "processing_date": []
}

topics = [
    "Что делать, если вам позвонили мошенники и предлагают перевести деньги на якобы специальный счет Центробанка",
    "Зачем нужны кредитные каникулы",
    "Что делать если убеждают взять кредит",
    "Стоит ли соглашаться если вам звонят и предлагают инвестировать",
    "Зачем нужны ипотечные каникулы",
    "Финансовые пирамиды",
    "Программа долгосрочных сбережений",
    "Как уберечь свои деньги от уловок мошенников",
    "Как мошенники пугают потерей номера мобильного телефона",
    "Что делать если карта заблокирована",
    "Случаи когда просят скачать программу на телефон и в чем же подвох",
    "Что делать если мошенники притворяются сотрудниками банка России",
    "Как сохранить деньги",
    "Мессенджеры",
    "Как заставить работать монеты",
    "Стоит ли верить, если советуют перевести деньги на безопасный счет",
    "Признаки финансовой пирамиды",
    "Как обратиться в банк России",
    "Что такое полная стоимость кредита",
    "Как сохранить деньги"
]


for i in range(1, 21):
    file_num = f"{i:02d}"
    filename = f"How_to_save_money_{file_num}.wav"
    filepath = os.path.join(audio_folder, filename)

    duration = get_wav_duration(filepath) if os.path.exists(filepath) else "00:00"

    metadata["filename"].append(filename)
    metadata["language"].append("чукотский")
    metadata["speaker_gender"].append("female")
    metadata["duration"].append(duration)
    metadata["topic"].append(topics[i-1])
    metadata["processing_date"].append(fixed_processing_date)

df = pd.DataFrame(metadata)
df.to_csv(output_csv, index=False, encoding='utf-8-sig')

print(f"Metadata for {len(df)} files saved to {output_csv}")

Metadata for 20 files saved to audio_metadata.csv
